> We have already established that using annotated HeLa m6A sites, we can observe changes in genes with m6A sites (HL-60) cells. In order to confirm this m6A sites, we performed MeRIP-seq in treated and untreated cells, and did observe a general increase in m6A levels upon treatments for a large number of annotated sites. Here, our goal is to indpendently analyze the MeRIP data without relying on HeLa annotations and use it to define a **treatment-induced hyper-methylation sites**. We will then assess the location and behaviour of these targets across the other datasets generated in this study.

## Test enrichment of treatment-induced hyper/hypo-methylation sites

### Goal
Here, I aim to identify the genes that are hyper or hypo methylated as genesets, and see if they have enriched accross all datasets; the input table is list of genes with control vs. treated fold change of RNA expression, RNA stability and translational efficiency. 
### Steps 
1. Prepare inputs  
    - Filtering genes with $\Delta$methylation >= 1 as hyper-methylation sites (P-Value < 0.01)
    - Filtering genes with $\Delta$methylation <= -1 as hypo-methylation sites (P-Value < 0.01)
2. Run `run_mi_gene_list.pl` command 


In [6]:
import pandas as pd 
import numpy as np


def two_sided_mtyl(fcthr=1,pvthr=0.01):
    delta_mtyl = pd.read_csv('../meRIP-seq/hl60_delta_mtyl_table.txt', sep='\t')
    ### hyper_methylation gene list
    # subset by threshold 
    hyper = delta_mtyl.iloc[np.where([(l and p) for l,p in zip(delta_mtyl.logFC >= fcthr,delta_mtyl.p_value < pvthr)])]
    ### hypo_methylation gene list
    # subset by threshold 
    hypo = delta_mtyl.iloc[np.where([(l and p) for l,p in zip(delta_mtyl.logFC <= -(fcthr),delta_mtyl.p_value < pvthr)])]
    
    return hyper, hypo

    
def write_gene_file(df,file_name):
    
    df = pd.DataFrame({'ensembl':[ens[:-3] for ens in df.ensembl.tolist()]}).drop_duplicates('ensembl')
    df.to_csv(file_name,sep='\t',index=None,header=None)
    

hyper, hypo = two_sided_mtyl()
write_gene_file(hyper,'mtyl_enrichment/hyper_mtyl.txt')
write_gene_file(hypo,'mtyl_enrichment/hypo_mtyl.txt')

2. Using a [TEISER](https://github.com/goodarzilab/TEISER) script to do enrichment test 



In [10]:
ls mtyl_enrichment

hyper_mtyl.txt  hypo_mtyl.txt  plots/


In [11]:
%%bash

declare -a Genesets=('hyper_mtyl' 'hypo_mtyl')
declare -a Experiments=(
# Ribo-seq
'../Ribo-seq/hl60_delta_te.txt'

## HL-60 RNA-seq 
# RNA experssion 
'../RNA-seq/hl60-exp/6h_delta_exp.txt' 
'../RNA-seq/hl60-exp/72h_delta_exp.txt' 
'../RNA-seq/hl60-exp/120h_delta_exp.txt' 
# RNA stability  
'../RNA-seq/hl60-stbl/120h_delta_stbl.txt'  
'../RNA-seq/hl60-stbl/6h_delta_stbl.txt'

## 5 other AML cell lines RNA-seq
# RNA experssion 
'../RNA-seq/other-exp/kg1_delta_exp.txt' 
'../RNA-seq/other-exp/molm14_delta_exp.txt'
'../RNA-seq/other-exp/ociaml2_delta_exp.txt' 
'../RNA-seq/other-exp/ociaml3_delta_exp.txt'
'../RNA-seq/other-exp/thp1_delta_exp.txt'
# RNA stability  
'../RNA-seq/other-stbl/kg1_delta_stbl.txt' 
'../RNA-seq/other-stbl/molm14_delta_stbl.txt' 
'../RNA-seq/other-stbl/ociaml2_delta_stbl.txt' 
'../RNA-seq/other-stbl/ociaml3_delta_stbl.txt'
'../RNA-seq/other-stbl/thp1_delta_stbl.txt'
)

for exp in "${Experiments[@]}"; do
    for geneset in "${Genesets[@]}"; do
    
        echo $exp $geneset
        base=`basename $exp`
        base=${base/.txt/}
        
        # get intersect 
        awk 'NR==FNR{A[$1];next}$1 in A' $exp mtyl_enrichment/${geneset}.txt > mtyl_enrichment/${geneset}_${base}.txt
        
        perl $TEISERDIR/run_mi_gene_list.pl \
            --expfile=$exp \
            --genefile=mtyl_enrichment/${geneset}_${base}.txt \
            --exptype=continuous \
            --ebins=7 \
            --species=human \
            --doremovedups=0 \
            --doremoveextra=0 &> mtyl_enrichment/${geneset}_${base}.log
#         # remove results from previous run 
#         rm -fr mtyl_enrichment/${geneset}_${base}_GENESET
        
        rm mtyl_enrichment/${geneset}_${base}.txt
        mv ${exp}_GENESET mtyl_enrichment/${geneset}_${base}_GENESET
        
        echo 'done!'
        
    done 

done

../Ribo-seq/hl60_delta_te.txt hyper_mtyl
done!
../Ribo-seq/hl60_delta_te.txt hypo_mtyl
done!
../RNA-seq/hl60-exp/6h_delta_exp.txt hyper_mtyl
done!
../RNA-seq/hl60-exp/6h_delta_exp.txt hypo_mtyl
done!
../RNA-seq/hl60-exp/72h_delta_exp.txt hyper_mtyl
done!
../RNA-seq/hl60-exp/72h_delta_exp.txt hypo_mtyl
done!
../RNA-seq/hl60-exp/120h_delta_exp.txt hyper_mtyl
done!
../RNA-seq/hl60-exp/120h_delta_exp.txt hypo_mtyl
done!
../RNA-seq/hl60-stbl/120h_delta_stbl.txt hyper_mtyl
done!
../RNA-seq/hl60-stbl/120h_delta_stbl.txt hypo_mtyl
done!
../RNA-seq/hl60-stbl/6h_delta_stbl.txt hyper_mtyl
done!
../RNA-seq/hl60-stbl/6h_delta_stbl.txt hypo_mtyl
done!
../RNA-seq/other-exp/kg1_delta_exp.txt hyper_mtyl
done!
../RNA-seq/other-exp/kg1_delta_exp.txt hypo_mtyl
done!
../RNA-seq/other-exp/molm14_delta_exp.txt hyper_mtyl
done!
../RNA-seq/other-exp/molm14_delta_exp.txt hypo_mtyl
done!
../RNA-seq/other-exp/ociaml2_delta_exp.txt hyper_mtyl
done!
../RNA-seq/other-exp/ociaml2_delta_exp.txt hypo_mtyl
done!
../RNA-

3. Redraw heatmaps using `--min=-3 --max=3` thresholds for those plots which have smaller range of signals:

In [14]:
%%bash 
declare -a Genesets=('hyper_mtyl' 'hypo_mtyl')
declare -a Experiments=(
'6h_delta_stbl' '120h_delta_stbl' 
'kg1_delta_stbl' 'ociaml2_delta_stbl' 'molm14_delta_stbl' 
'ociaml3_delta_stbl' 'thp1_delta_stbl'
'hl60_delta_te'
)
for exp in "${Experiments[@]}"; do
    for geneset in "${Genesets[@]}"; do
        echo $exp $geneset    
        perl $TEISERDIR/Scripts/teiser_draw_matrix.pl \
        --pvmatrixfile=mtyl_enrichment/${geneset}_${exp}_GENESET/${exp}.txt.matrix \
        --summaryfile=mtyl_enrichment/${geneset}_${exp}_GENESET/${exp}.txt.summary \
        --expfile=mtyl_enrichment/${geneset}_${exp}_GENESET/${exp}.txt \
        --quantized=0 \
        --colmap=$TEISERDIR/Scripts/HEATMAPS/cmap_1.txt \
        --order=0 \
        --min=-3 --max=3 \
        --cluster=5 &>> mtyl_enrichment/${geneset}_${exp}.log
        
        echo 'done!'
        
    done 
done

6h_delta_stbl hyper_mtyl
done!
6h_delta_stbl hypo_mtyl
done!
120h_delta_stbl hyper_mtyl
done!
120h_delta_stbl hypo_mtyl
done!
kg1_delta_stbl hyper_mtyl
done!
kg1_delta_stbl hypo_mtyl
done!
ociaml2_delta_stbl hyper_mtyl
done!
ociaml2_delta_stbl hypo_mtyl
done!
molm14_delta_stbl hyper_mtyl
done!
molm14_delta_stbl hypo_mtyl
done!
ociaml3_delta_stbl hyper_mtyl
done!
ociaml3_delta_stbl hypo_mtyl
done!
thp1_delta_stbl hyper_mtyl
done!
thp1_delta_stbl hypo_mtyl
done!
hl60_delta_te hyper_mtyl
done!
hl60_delta_te hypo_mtyl
done!


4. Make `png` figures:

In [17]:
mkdir -p mtyl_enrichment/plots/

In [19]:
%%bash 
for pdf in mtyl_enrichment/*_GENESET/*.txt.summary.pdf; do 
    png=${pdf/.pdf/.png}
    di=`dirname $pdf`
    out=`basename $di`
    outpng=${out/_GENESET/.png}
    outpdf=${out/_GENESET/.pdf}
    
    bash /rumi/shams/abe/GitHub/Abe/my_scripts/pdf2png.sh $pdf
    cp $pdf mtyl_enrichment/plots/$outpdf
    mv $png mtyl_enrichment/plots/$outpng
done 

mtyl_enrichment/hyper_mtyl_120h_delta_exp_GENESET/120h_delta_exp.txt.summary.pdf > mtyl_enrichment/hyper_mtyl_120h_delta_exp_GENESET/120h_delta_exp.txt.summary.png
done!
mtyl_enrichment/hyper_mtyl_120h_delta_stbl_GENESET/120h_delta_stbl.txt.summary.pdf > mtyl_enrichment/hyper_mtyl_120h_delta_stbl_GENESET/120h_delta_stbl.txt.summary.png
done!
mtyl_enrichment/hyper_mtyl_6h_delta_exp_GENESET/6h_delta_exp.txt.summary.pdf > mtyl_enrichment/hyper_mtyl_6h_delta_exp_GENESET/6h_delta_exp.txt.summary.png
done!
mtyl_enrichment/hyper_mtyl_6h_delta_stbl_GENESET/6h_delta_stbl.txt.summary.pdf > mtyl_enrichment/hyper_mtyl_6h_delta_stbl_GENESET/6h_delta_stbl.txt.summary.png
done!
mtyl_enrichment/hyper_mtyl_72h_delta_exp_GENESET/72h_delta_exp.txt.summary.pdf > mtyl_enrichment/hyper_mtyl_72h_delta_exp_GENESET/72h_delta_exp.txt.summary.png
done!
mtyl_enrichment/hyper_mtyl_hl60_delta_te_GENESET/hl60_delta_te.txt.summary.pdf > mtyl_enrichment/hyper_mtyl_hl60_delta_te_GENESET/hl60_delta_te.txt.summary.png
do

5. Write README.md draft
    - Write HTML codes which link all plots into a `README.md` format to prepare GitHub friendly report

In [25]:
%%bash 
readme='mtyl_enrichment.md'
touch $readme
for f in mtyl_enrichment/plots/*.png; do 
    b=`basename $f`
    t=${b/.png/}
    echo '#### '$t >> $readme
    echo -e "<img src=\""$f"\" title=\""$t"\" style=\"width:1000px\">\n" >> $readme
done 

## Hyper-geometric test

### Goal
Here, I aim to do _hyper-geometric test_ on enriched genesets (iPAGE results) in CRISPR screening experiment (Rho scores) to test hyper/hypo methylation enrichment. 

___
- Clean up iPAGE results with no signal

1. Read iPAGE results into python

In [8]:
ls CRISPRi-screen/hl60_exp2

hl60_DAC_processing_output/
hl60_DAC_processing_output_genetable_collapsed.txt
hl60_GSK_processing_output/
hl60_GSK_processing_output_genetable_collapsed.txt


In [7]:
ls CRISPRi-screen/hl60_exp1

hl60_DAC_processing_output_gamma/
hl60_DAC_processing_output_gamma.txt
hl60_DAC_processing_output_genetable_collapsed.xlsx
hl60_DAC_processing_output_rho/
hl60_DAC_processing_output_rho.txt
ipage.out
README.md


In [ ]:
# https://bioinformatics.stackexchange.com/questions/5400/how-to-convert-data-in-gmt-format-to-dataframe
# https://gseapy.readthedocs.io/en/latest/gseapy_tutorial.html

In [28]:
import pandas as pd
from glob import glob 
import sys

sys.path.append('/rumi/shams/abe/GitHub/Abe/my_scripts/')

import ipage_down as ipd

In [32]:
type(ipd.make_page_dict)

function

In [31]:
[
    'make_page_dict',
    'read_gmt',
    'read_page_annotations',
    'read_page_index',
    'read_page_names',
    'subset_dict'
]

['make_page_dict',
 'read_gmt',
 'read_page_annotations',
 'read_page_index',
 'read_page_names',
 'subset_dict']

In [2]:
def get_enriched(pagefolder,side='L'):
    paths = glob(f'{pagefolder}/*/pvmatrix.{side}.txt')

    out = {} 
    for f in paths:
        page = make_page_dict(f)
        if 'index' in page['annotations'].keys():
            key = page['gs_name']
#             out[key] = page['data'].index.tolist()
            out[key] = [page['annotations']['index'][gs] for gs in page['data'].index.tolist()]
        
    return out

In [ ]:
D = read_page_index('/flash/bin/iPAGEv1.0/PAGE_DATA/ANNOTATIONS/human_c2_gs/human_c2_gs_index.txt')

In [ ]:
enrich = get_enriched('screen/CRISPRi_HL60_rho/')

In [ ]:
import numpy as np

In [ ]:
out.values()

In [ ]:
l_page = [make_page_dict(path) for path in glob('screen/CRISPRi_HL60_rho/*/pvmatrix.L.txt')]
r_page = [make_page_dict(path) for path in glob('screen/CRISPRi_HL60_rho/*/pvmatrix.R.txt')]

2. Run the hypergeom test





https://github.com/JohnDeJesus22/DataScienceMathFunctions/blob/master/hypergeometricfunctions.py#L38

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import comb

def hypergeom_pmf(N, A, n, x):
    '''
    Probability Mass Function for Hypergeometric Distribution
    :param N: population size
    :param A: total number of desired items in N
    :param n: number of draws made from N
    :param x: number of desired items in our draw of n items
    :returns: PMF computed at x
    '''
    Achoosex = comb(A,x)
    NAchoosenx = comb(N-A, n-x)
    Nchoosen = comb(N,n)
    
    return (Achoosex)*NAchoosenx/Nchoosen
    
    
def hypergeom_plot(N, A, n):
    '''
    Visualization of Hypergeometric Distribution for given parameters
    '''
    x = np.arange(0, n+1)
    y = [hypergeom_pmf(N, A, n, x) for x in range(n+1)]
    plt.plot(x, y, 'bo')
    plt.vlines(x, 0, y, lw=2)
    plt.xlabel('# of desired items in our draw')
    plt.ylabel('Probablities')
    plt.title('Hypergeometric Distribution Plot')
    plt.show()


In [ ]:
hyper, hypo = [set(list(mtyl.name)) for mtyl in two_sided_mtyl()]

In [ ]:
l_page

In [ ]:
gmt = l_page[3]['annotations']['gmt']
pw = [gmt[pw]['genes'] for pw in gmt][0]

In [ ]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

In [ ]:
N = len(hyper)
A = len(intersection(hyper, pw))

In [ ]:
help(hypergeom_pmf)

For this matter, we have:

    ○ N = # of all genes 
    ○ n/A = # of metylated genes OR enriched pathway's genes 
    ○ x = # of genes in n and A overlap 

In [ ]:
hypergeom_pmf(100,10,5,4)

In [ ]:
from utility import *

In [ ]:
screen = pd.read_excel('screen/CRISPRi_HL60_DAC_genetable_collapsed.xlsx')

Data['hl60']['rho'] = make_score_df(screen, 'rho')
Data['hl60']['gamma'] = make_score_df(screen, 'gamma')

OK! I need to write something that can read index and names in iPAGE format, easy!